# Megaline Plan Recommendation Model

In [1]:
#Importar librerias, cargar y examinacion de datos
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv('/datasets/users_behavior.csv')

print(data.head())
print(data.info())
print(data.describe())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246  

In [2]:
#Preparar caracteristicas y objetivo
features = data[['calls', 'minutes', 'messages', 'mb_used']]
target = data['is_ultra']

In [3]:
#Dividir los datos en 60% train, 20% validation y 20% test, (0.25 de 0.8 = 0.2)
features_temp, features_test, target_temp, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_temp, target_temp, test_size=0.25, random_state=12345)


print('Tamaño de train:', features_train.shape)
print('Tamaño de validacion:', features_valid.shape)
print('Tamaño de prueba:', features_test.shape)

Tamaño de train: (1928, 4)
Tamaño de validacion: (643, 4)
Tamaño de prueba: (643, 4)


### ¿Segmentaste correctamente los datos en conjuntos de entrenamiento, validación y prueba?
Si, se dividieron los datos en tres partes: entrenamiento (60%), validación (20%) y prueba (20%), utilizando la funcion train_test_split en dos pasos.



### ¿Cómo escogiste el tamaño de los conjuntos?
Primero se separo el 60% para entrenamiento y el 40% restante se dividió en dos mitades iguales: 20% para validacion y 20% para prueba, ya que este tamaño permite entrenar el modelo con suficientes datos y tener conjuntos independientes para evaluar y ajustar.

In [4]:
#Entrenamos modelos

#1 decision tree
best_tree_accuracy = 0
best_tree_depth = 0

for depth in range(1, 21):
    model_tree = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model_tree.fit(features_train, target_train)
    preds_valid = model_tree.predict(features_valid)
    acc = accuracy_score(target_valid, preds_valid)
    if acc > best_tree_accuracy:
        best_tree_accuracy = acc
        best_tree_depth = depth

print(f'Mejor exactitud arbol de decision: {best_tree_accuracy:.3f} con profundidad {best_tree_depth}')

#2 random forest
best_forest_accuracy = 0
best_forest_estimators = 0

for est in range(10, 101, 10):
    model_forest = RandomForestClassifier(n_estimators=est, random_state=12345)
    model_forest.fit(features_train, target_train)
    preds_valid = model_forest.predict(features_valid)
    acc = accuracy_score(target_valid, preds_valid)
    if acc > best_forest_accuracy:
        best_forest_accuracy = acc
        best_forest_estimators = est

print(f'Mejor exactitud bosque aleatorio: {best_forest_accuracy:.3f} con {best_forest_estimators} arboles')

#3 regresion logistica
model_logreg = LogisticRegression(random_state=12345, solver='liblinear')
model_logreg.fit(features_train, target_train)
preds_valid = model_logreg.predict(features_valid)
logreg_accuracy = accuracy_score(target_valid, preds_valid)

print(f'Exactitud regresion logistica: {logreg_accuracy:.3f}')

Mejor exactitud arbol de decision: 0.774 con profundidad 7
Mejor exactitud bosque aleatorio: 0.799 con 80 arboles
Exactitud regresion logistica: 0.729


### ¿Evaluaste correctamente la calidad del modelo?
Si, la calidad del modelo fue evaluada utilizando la metrica de exactitud (accuracy) en el conjunto de prueba.

In [5]:
#Seleccionamos el mejor modelo
if best_forest_accuracy >= best_tree_accuracy and best_forest_accuracy >= logreg_accuracy:
    final_model = RandomForestClassifier(n_estimators=best_forest_estimators, random_state=12345)
    print('Modelo seleccionado: Random Forest')
elif best_tree_accuracy >= best_forest_accuracy and best_tree_accuracy >= logreg_accuracy:
    final_model = DecisionTreeClassifier(max_depth=best_tree_depth, random_state=12345)
    print('Modelo seleccionado: Decision Tree')
else:
    final_model = LogisticRegression(random_state=12345, solver='liblinear')
    print('Modelo seleccionado: Logistic Regression')

Modelo seleccionado: Random Forest


### ¿Qué modelos e hiperparámetros usaste?
Probe los siguientes modelos:
1. DecisionTreeClassifier
2. RandomForestClassifier
3. LogisticRegression


### ¿Cuáles fueron tus hallazgos?
1. El Random Forest fue el mejor modelo en terminos de exactitud como tal.
2. El arbol de decision tuvo una exactitud decente pero menor que Random Forest por lo que fue descartado.
3. La regresion logistica quedo por debajo de los arboles en desempeño por ende tambien fue descartado.
4. El mejor desempeño se logró alcanzando una exactitud de 0.787 en el conjunto de prueba suoerando el umbral requerido de 0.75 de exactitud.

### ¿Probaste los modelos correctamente?
Desde mi perspectiva si, ya que realice las pruebas de todos modelos usando el conjunto de validacion antes de hacer la evaluacion final en el conjunto de prueba.

In [6]:
# 7. Entrenar el modelo final y evaluar en test
final_model.fit(features_train, target_train)
preds_test = final_model.predict(features_test)
final_accuracy = accuracy_score(target_test, preds_test)

print(f'Exactitud final en conjunto de prueba: {final_accuracy:.3f}')

Exactitud final en conjunto de prueba: 0.787


### ¿Cuál es tu puntuación de exactitud?

La exactitud final en el conjunto de prueba fue de 0.787 (78.7%).

In [7]:
#Hacemos la prueba de cordura usando dummy classifier
dummy = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy.fit(features_train, target_train)
dummy_preds = dummy.predict(features_test)
dummy_accuracy = accuracy_score(target_test, dummy_preds)

print(f'Exactitud del modelo aleatorio (dummy): {dummy_accuracy:.3f}')

Exactitud del modelo aleatorio (dummy): 0.695


In [8]:
#Conclusion
if final_accuracy > dummy_accuracy:
    print('El modelo es mejor que uno aleatorio.')
else:
    print('El modelo no supera a uno aleatorio. Hay que revisar el enfoque.')

El modelo es mejor que uno aleatorio.


# Conclusion 

Desarrolle un modelo de clasificacion para recomendar el plan mas adecuado (Smart o Ultra) a los clientes de Megaline.

Ahora bien, despues de un analisis de resultados seleccione utilize el modelo Random Forest, el modelo final logro una exactitud de un 78.1%, superando el umbral minimo establecido del 75%.

El modelo fue de ayuda para evitar el sobreajuste mediante una separación eficiente de los datos y una evaluacion transparente, ya que con estos resultados, desde mi perspectiva el modelo puede ser utilizado para asistir en la recomendacion de planes a los clientes de manera confiable.